# Homework Number 4 Information Retrieval
Due October 8, 2019

In [1]:
import pandas as pd
import numpy as np
import re
import nltk 
from nltk.tokenize import word_tokenize
from numpy import dot
from numpy.linalg import norm
import collections


## Part 1. Read files
### 1. read cran.qry -> queries
- contains a set of 225 queries numbered 001 through 365, but referred to in cranqrel below as 1 through 225 
- Lines beginning with .I are ids for the queries (001 to 365)
- Lines following .W are the queries

In [2]:
queries = []

f= open("cran.qry","r")
line = f.readline()

for i in range(225):
    #print("this is loop",i)
    index = line[3:-1]
    #print(index)
    
    line = f.readline() #this should be .W
    line = f.readline() #this should be the first line of query
    query = ''
    while(line != '' and line[0]!='.'):
        query = query + line.strip('\n') + " "
        line = f.readline()
        #print(query)
    query = query[:-3]
    queries.append([i+1, query])

f.close()

queries[-1]

[225,
 'what design factors can be used to control lift-drag ratios at mach numbers above 5']

### 2. cran.all.1400 -- contains 1400 abstracts of aerodynamics journal articles (the document collection)

- Lines beginning with .I are ids for the abstracts
- Lines following .T are titles
- Lines following .A are authors
- Lines following .B are some sort of bibliographic notation
- Lines following .W are the abstracts

In [3]:
articles = []

f= open("cran.all.1400","r")
line = f.readline()

for i in range(1400):
    #print("this is loop",i)
    index = line[3:-1]
    #print(index)
    
    line = f.readline()
    while line[0:2] != '.W':
        line = f.readline()
    
    #now line starts with '.W'
    line = f.readline() #first line of actual article
    content = ''
    while(line != '' and line[0:2]!='.I'):
        content = content + line.strip('\n') + " "
        line = f.readline()
        #print(content)

    content = content[:-3]
    articles.append([i+1, content])

f.close()

articles[-1]

[1400,
 'the buckling shear stress of simply-supported infinitely long plates with transverse stiffeners .   this report is an extension of previous theoretical investigations of the elastic buckling in shear of flat plates reinforced by transverse stiffeners . the plates are treated as infinitely long and simply-supported along the long sides . stiffeners are spaced at regular intervals, dividing the plate into a number of panels of uniform size . the effect ob bending and torsional stiffnesses of the stiffener upon the buckling shear stress is calculated for the complete range of stiffnesses, for panels with ratios of width to stiffener spacing of graphical forms']

In [4]:
#stop_words copy paste from stop_list.py file
stop_words = ['a','the','an','and','or','but','about','above','after','along','amid','among',\
                        'as','at','by','for','from','in','into','like','minus','near','of','off','on',\
                           'onto','out','over','past','per','plus','since','till','to','under','until','up',\
                           'via','vs','with','that','can','cannot','could','may','might','must',\
                           'need','ought','shall','should','will','would','have','had','has','having','be',\
                           'is','am','are','was','were','being','been','get','gets','got','gotten',\
                           'getting','seem','seeming','seems','seemed',\
                           'enough', 'both', 'all', 'your' 'those', 'this', 'these', \
                           'their', 'the', 'that', 'some', 'our', 'no', 'neither', 'my',\
                           'its', 'his' 'her', 'every', 'either', 'each', 'any', 'another',\
                           'an', 'a', 'just', 'mere', 'such', 'merely' 'right', 'no', 'not',\
                           'only', 'sheer', 'even', 'especially', 'namely', 'as', 'more',\
                           'most', 'less' 'least', 'so', 'enough', 'too', 'pretty', 'quite',\
                           'rather', 'somewhat', 'sufficiently' 'same', 'different', 'such',\
                           'when', 'why', 'where', 'how', 'what', 'who', 'whom', 'which',\
                           'whether', 'why', 'whose', 'if', 'anybody', 'anyone', 'anyplace', \
                           'anything', 'anytime' 'anywhere', 'everybody', 'everyday',\
                           'everyone', 'everyplace', 'everything' 'everywhere', 'whatever',\
                           'whenever', 'whereever', 'whichever', 'whoever', 'whomever' 'he',\
                           'him', 'his', 'her', 'she', 'it', 'they', 'them', 'its', 'their','theirs',\
                           'you','your','yours','me','my','mine','I','we','us','much','and/or'
                           ]



In [5]:
#removing stop words, punctuation and numbers
def clean_up(dump):
    new_dump = []
    for i in range(len(dump)): #for each query or article
        #print("$$$$",dump[i][1])
        #words is a list of words <- dump[i][1]
        words = dump[i][1].split(' ')
        
        #re.match(pattern, string, flags=0)
        #only consider all-characters words -> neglect punctuation and numbers
        words = [x for x in words if re.match('[A-z]+',x)]

        #remove stop words
        words = [x for x in words if x not in stop_words]
        new_dump.append(words)
    return new_dump

In [6]:
cleaned_queries = clean_up(queries)
cleaned_queries[0]

['similarity',
 'laws',
 'obeyed',
 'constructing',
 'aeroelastic',
 'models',
 'heated',
 'high',
 'speed',
 'aircraft']

In [7]:
cleaned_articles = clean_up(articles)
cleaned_articles[0]

['experimental',
 'investigation',
 'aerodynamics',
 'wing',
 'slipstream',
 'experimental',
 'study',
 'wing',
 'propeller',
 'slipstream',
 'made',
 'order',
 'determine',
 'spanwise',
 'distribution',
 'lift',
 'increase',
 'due',
 'slipstream',
 'angles',
 'attack',
 'wing',
 'free',
 'stream',
 'slipstream',
 'velocity',
 'ratios',
 'results',
 'intended',
 'part',
 'evaluation',
 'basis',
 'theoretical',
 'treatments',
 'problem',
 'comparative',
 'span',
 'loading',
 'curves,',
 'together',
 'supporting',
 'evidence,',
 'showed',
 'substantial',
 'part',
 'lift',
 'increment',
 'produced',
 'slipstream',
 'due',
 'boundary-layer-control',
 'effect',
 'integrated',
 'remaining',
 'lift',
 'increment,',
 'subtracting',
 'destalling',
 'lift,',
 'found',
 'agree',
 'well',
 'potential',
 'flow',
 'theory',
 'empirical',
 'evaluation',
 'destalling',
 'effects',
 'made',
 'specific',
 'configuration',
 'experiment']

## Part 2. For each query, create feature vector -> compute TF-IDF score

In [8]:
#get frequencies of each query/article and overall frequencies of whole queries/articles
'''
def frequencies(dump):
    all_freq = {}
    dump_freq = []
    for i in range(len(dump)): #for each query
        scores = {}
        for w in dump[i]:
            if w not in scores: #first encounter this word in this query
                scores.update({w:1})
                if w not in all_freq: #update in the huge_dict for idf
                    all_freq.update({w:1})
                else:
                    all_freq.update({w:all_freq[w]+1})
            else:
                scores[w] += 1
        dump_freq.append(scores)
    return all_freq, dump_freq
'''

'\ndef frequencies(dump):\n    all_freq = {}\n    dump_freq = []\n    for i in range(len(dump)): #for each query\n        scores = {}\n        for w in dump[i]:\n            if w not in scores: #first encounter this word in this query\n                scores.update({w:1})\n                if w not in all_freq: #update in the huge_dict for idf\n                    all_freq.update({w:1})\n                else:\n                    all_freq.update({w:all_freq[w]+1})\n            else:\n                scores[w] += 1\n        dump_freq.append(scores)\n    return all_freq, dump_freq\n'

In [9]:
#get frequencies of each query/article and overall frequencies of whole queries/articles
#improved frequencies method = Use 1 if the term exists in the document and 0 if it does not
def frequencies(dump):
    all_freq = {}
    dump_freq = []
    for i in range(len(dump)): #for each query
        scores = {}
        for w in dump[i]:
            if w not in scores: #first encounter this word in this query
                scores.update({w:1})
                if w not in all_freq: #update in the huge_dict for idf
                    all_freq.update({w:1})
                
                #else:
                    #all_freq.update({w:all_freq[w]+1})
            #else:
                #scores[w] += 1
        dump_freq.append(scores)
    return all_freq, dump_freq


In [10]:
all_queries_freq, each_queries_freq = frequencies(cleaned_queries)
all_articles_freq, each_articles_freq = frequencies(cleaned_articles)

In [11]:
all_queries_freq

{'similarity': 1,
 'laws': 1,
 'obeyed': 1,
 'constructing': 1,
 'aeroelastic': 1,
 'models': 1,
 'heated': 1,
 'high': 1,
 'speed': 1,
 'aircraft': 1,
 'structural': 1,
 'problems': 1,
 'associated': 1,
 'flight': 1,
 'heat': 1,
 'conduction': 1,
 'composite': 1,
 'slabs': 1,
 'solved': 1,
 'far': 1,
 'criterion': 1,
 'developed': 1,
 'show': 1,
 'empirically': 1,
 'validity': 1,
 'flow': 1,
 'solutions': 1,
 'chemically': 1,
 'reacting': 1,
 'gas': 1,
 'mixtures': 1,
 'based': 1,
 'simplifying': 1,
 'assumption': 1,
 'instantaneous': 1,
 'local': 1,
 'chemical': 1,
 'equilibrium': 1,
 'kinetic': 1,
 'system': 1,
 'applicable': 1,
 'hypersonic': 1,
 'aerodynamic': 1,
 'theoretical': 1,
 'experimental': 1,
 'guides': 1,
 'do': 1,
 'turbulent': 1,
 'couette': 1,
 'behaviour': 1,
 'possible': 1,
 'relate': 1,
 'available': 1,
 'pressure': 1,
 'distributions': 1,
 'ogive': 1,
 'forebody': 1,
 'zero': 1,
 'angle': 1,
 'attack': 1,
 'lower': 1,
 'surface': 1,
 'pressures': 1,
 'equivalent':

In [12]:
each_queries_freq

[{'similarity': 1,
  'laws': 1,
  'obeyed': 1,
  'constructing': 1,
  'aeroelastic': 1,
  'models': 1,
  'heated': 1,
  'high': 1,
  'speed': 1,
  'aircraft': 1},
 {'structural': 1,
  'aeroelastic': 1,
  'problems': 1,
  'associated': 1,
  'flight': 1,
  'high': 1,
  'speed': 1,
  'aircraft': 1},
 {'problems': 1,
  'heat': 1,
  'conduction': 1,
  'composite': 1,
  'slabs': 1,
  'solved': 1,
  'far': 1},
 {'criterion': 1,
  'developed': 1,
  'show': 1,
  'empirically': 1,
  'validity': 1,
  'flow': 1,
  'solutions': 1,
  'chemically': 1,
  'reacting': 1,
  'gas': 1,
  'mixtures': 1,
  'based': 1,
  'simplifying': 1,
  'assumption': 1,
  'instantaneous': 1,
  'local': 1,
  'chemical': 1,
  'equilibrium': 1},
 {'chemical': 1,
  'kinetic': 1,
  'system': 1,
  'applicable': 1,
  'hypersonic': 1,
  'aerodynamic': 1,
  'problems': 1},
 {'theoretical': 1,
  'experimental': 1,
  'guides': 1,
  'do': 1,
  'turbulent': 1,
  'couette': 1,
  'flow': 1,
  'behaviour': 1},
 {'possible': 1,
  'relate'

In [13]:
def idf (all_freq, each_freq):
    num = len(each_freq)
    all_idf = {}
    for i in range(num):
        for w in each_freq[i]:
            if w in all_idf:
                all_idf.update({w: all_idf[w]+1})
            else:
                all_idf.update({w: 1})
        
    all_idf = {k:np.log(num/v) for k, v in all_idf.items()}
    return all_idf

In [14]:
all_idf = idf(all_articles_freq, each_articles_freq)

In [15]:
#tfidf according to frequencies
def tfidf(dump, all_idf):
    num = len(dump)
    all_dump_freq, each_dump_freq = frequencies(dump)
   
    
    tfidf = []
    for i in range(len(dump)):
        tfidf.append({})
        
        
        for k, v in each_dump_freq[i].items():
            if k in all_idf:
                temp = {k: np.log(v/len(each_dump_freq[i]))*all_idf[k]}
            else:
                temp = {k:0}
            tfidf[i].update(temp)
    return tfidf

In [16]:
#feature vector
queries_tfidf = tfidf(cleaned_queries, all_idf)

In [17]:
queries_tfidf

[{'similarity': -7.766680547985573,
  'laws': -11.378552177207164,
  'obeyed': 0,
  'constructing': -12.974582542415346,
  'aeroelastic': -10.444934263548086,
  'models': -7.291573353223209,
  'heated': -9.563062012770962,
  'high': -4.6977820661812055,
  'speed': -5.754840301647834,
  'aircraft': -6.9310597680507895},
 {'structural': -7.670809179133754,
  'aeroelastic': -9.432715548199347,
  'problems': -4.926259795574905,
  'associated': -6.351805517460052,
  'flight': -5.2152998300946205,
  'high': -4.242519945038571,
  'speed': -5.197138653155855,
  'aircraft': -6.259370675769371},
 {'problems': -4.609919894813899,
  'heat': -3.5573711037261715,
  'conduction': -7.23463511059971,
  'composite': -9.821014239885997,
  'slabs': -10.610012908841485,
  'solved': -6.523483228285373,
  'far': -7.070094031058294},
 {'criterion': -12.427987350642928,
  'developed': -6.128349844329796,
  'show': -7.432298126262675,
  'empirically': -20.938510618874222,
  'validity': -11.634762842500454,
  'f

## Part 3. Compute the IDF scores for each word in the collection of abstracts

In [18]:

articles_idf = all_idf
articles_tfidf = tfidf(cleaned_articles, all_idf)

In [19]:
articles_idf

{'experimental': 1.4884853020164377,
 'investigation': 1.8971199848858813,
 'aerodynamics': 4.248495242049359,
 'wing': 2.1322397272468065,
 'slipstream': 4.75932086581535,
 'study': 2.324246589775225,
 'propeller': 4.411014171547134,
 'made': 1.4006830985719902,
 'order': 2.3097535824726583,
 'determine': 2.6291069987620905,
 'spanwise': 4.02535169073515,
 'distribution': 1.8061482066801546,
 'lift': 2.473542891137685,
 'increase': 2.659260036932778,
 'due': 2.2954676252251818,
 'angles': 2.7898802193498424,
 'attack': 2.711628022450094,
 'free': 2.6094985273737143,
 'stream': 2.361425593016979,
 'velocity': 1.7068932485848134,
 'ratios': 2.6798793241355137,
 'results': 0.8777570678719121,
 'intended': 4.75932086581535,
 'part': 2.849778360930911,
 'evaluation': 4.0661736852554045,
 'basis': 3.069840245707713,
 'theoretical': 1.8689491079191851,
 'treatments': 4.846332242804979,
 'problem': 1.8370557441432314,
 'comparative': 5.452468046375295,
 'span': 4.02535169073515,
 'loading': 2

In [20]:
articles_tfidf

[{'experimental': -6.069354553675338,
  'investigation': -7.735577773954034,
  'aerodynamics': -17.323398429711556,
  'wing': -8.694287327232173,
  'slipstream': -19.40630903792388,
  'study': -9.477202498678658,
  'propeller': -17.986075450082204,
  'made': -5.711337781473176,
  'order': -9.418106718727643,
  'determine': -10.720282231487014,
  'spanwise': -16.41352224386177,
  'distribution': -7.364636941981497,
  'lift': -10.085963757720721,
  'increase': -10.843232373675509,
  'due': -9.35985519312902,
  'angles': -11.375841058410884,
  'attack': -11.056764795484277,
  'free': -10.640327955183155,
  'stream': -9.628801276524001,
  'velocity': -6.959921133854451,
  'ratios': -10.927308289311311,
  'results': -3.579087310900616,
  'intended': -19.40630903792388,
  'part': -11.620077973528058,
  'evaluation': -16.579975455053024,
  'basis': -12.517388548681934,
  'theoretical': -7.620709968294669,
  'treatments': -19.76110118564489,
  'problem': -7.4906635832861115,
  'comparative': -

## Part 4. Count the number of instances of each non-stop-word in each abstract
- this is in all_articles_freq and each_articles_freq

## Part 5. 
For each query:

- For each abstract:

----Create a vector representing scores for words in the query, based on their TF-IDF values in the abstract, e.g., if the only words contained in both query and abstract are "chicken" and "fish", then values in the vector representing these words would have non-zero values and the other values in the vector would be zero. The vector would be the same length as the query's vector.

----Calculate the cosine similarity between vectors for query and abstract

- Sort the abstracts by cosine similarity scores. So for each query there should be a ranking of all 1400 documents (see the sample output file which contains 225*1400=315,000 lines)


In [ ]:
result = open('output.txt','w+')

for i in range(225):
    print('now we are in query: ', i)
    cosine_scores = {}
    query = cleaned_queries[i]
    
    for j in range(1400): 
        article = cleaned_articles[j]
        
        q_vec = np.empty(0)
        a_vec = np.empty(0)
        
        for w in query:
            q_vec = np.append(q_vec, queries_tfidf[i][w])
            if w in article:
                a_vec = np.append(a_vec, articles_tfidf[j][w])
            else:
                a_vec = np.append(a_vec, 0)
        if i==0 and j==34:
            
            print(i,j,q_vec,a_vec)
        
        norm_q = norm(q_vec)
        norm_a = norm(a_vec)
        if norm_q != 0 and norm_a != 0:
            cos_sim = dot(q_vec,a_vec)/(norm_q*norm_a)
            cosine_scores.update({j+1 : cos_sim})
        
        removed_s = {k: v for k, v in cosine_scores.items() if v > 0.25}
    '''
    removed_s = {}
    for k,v in cosine_scores.items():
        if v > 0.2:
            removed_s.update({k,v})
    '''
    
    sorted_s = sorted(removed_s.items(), key=lambda kv: kv[1], reverse=True)    
    
    
    for k,v in sorted_s:
        result.write('%d %d %f'% (i+1,k,v))
        result.write('\n')
    
result.close()        
#cos_sim = dot(a, b)/(norm(a)*norm(b))

now we are in query:  0
0 34 [ -7.76668055 -11.37855218   0.         -12.97458254 -10.44493426
  -7.29157335  -9.56306201  -4.69778207  -5.7548403   -6.93105977] [  0.           0.           0.         -23.85829931   0.
   0.           0.           0.           0.           0.        ]
now we are in query:  1
now we are in query:  2
now we are in query:  3
now we are in query:  4
now we are in query:  5
now we are in query:  6
now we are in query:  7
now we are in query:  8
now we are in query:  9
now we are in query:  10
now we are in query:  11
now we are in query:  12
now we are in query:  13
now we are in query:  14
now we are in query:  15
now we are in query:  16
now we are in query:  17
now we are in query:  18
now we are in query:  19
now we are in query:  20
now we are in query:  21
now we are in query:  22
now we are in query:  23
now we are in query:  24
now we are in query:  25
now we are in query:  26
now we are in query:  27
now we are in query:  28
now we are in query:  